In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1753851124424_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Start Spark Session
spark = SparkSession.builder.appName("MergeNYCData").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
clean_sample_df= spark.read.parquet("s3://nycfinalp/part-00000-664c0e47-18b2-4b29-9fbc-51eaed404954-c000.snappy.parquet")
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|DOLocationID|mta_tax|trip_distance|PULocationID|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|
+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|                 N|            1.0|  0.5|       0.0|       10.5|        11.8|          79|    0.5|         1.76|          68|       2| 2016-01-01 00:38:14|  2016-01-01 00:52:44|                  0.3|           2|       1.0|         0.0|2016|
|                 N|        

In [6]:
# Row count
row_count = clean_sample_df.count()

# Column count
column_count = len(clean_sample_df.columns)

print(f"Row count: {row_count}")
print(f"Column count: {column_count}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row count: 992925
Column count: 18

In [7]:
from pyspark.sql import SparkSession

# Create Spark session (skip if already created)
spark = SparkSession.builder.appName("TaxiZoneLookup").getOrCreate()

# Path to the zone lookup file in S3
zone_lookup_path = "s3://nycfinalp/taxi_zone_lookup.csv"

# Read the CSV file from S3
zone_df = spark.read.option("header", True).csv(zone_lookup_path)

# Create renamed DataFrames for joining
pickup_zone_df = zone_df.withColumnRenamed("LocationID", "PULocationID") \
                        .withColumnRenamed("Zone", "pickup_zone")

dropoff_zone_df = zone_df.withColumnRenamed("LocationID", "DOLocationID") \
                         .withColumnRenamed("Zone", "dropoff_zone")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|DOLocationID|mta_tax|trip_distance|PULocationID|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|
+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|                 N|            1.0|  0.5|       0.0|       10.5|        11.8|          79|    0.5|         1.76|          68|       2| 2016-01-01 00:38:14|  2016-01-01 00:52:44|                  0.3|           2|       1.0|         0.0|2016|
|                 N|        

In [10]:
sample_with_zones = clean_sample_df \
    .join(pickup_zone_df, on="PULocationID", how="left") \
    .join(dropoff_zone_df, on="DOLocationID", how="left")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|DOLocationID|mta_tax|trip_distance|PULocationID|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|
+------------------+---------------+-----+----------+-----------+------------+------------+-------+-------------+------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+
|                 N|            1.0|  0.5|       0.0|       10.5|        11.8|          79|    0.5|         1.76|          68|       2| 2016-01-01 00:38:14|  2016-01-01 00:52:44|                  0.3|           2|       1.0|         0.0|2016|
|                 N|        

In [12]:
pickup_zone_df.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-----------------------+------------+
|PULocationID|Borough      |pickup_zone            |service_zone|
+------------+-------------+-----------------------+------------+
|1           |EWR          |Newark Airport         |EWR         |
|2           |Queens       |Jamaica Bay            |Boro Zone   |
|3           |Bronx        |Allerton/Pelham Gardens|Boro Zone   |
|4           |Manhattan    |Alphabet City          |Yellow Zone |
|5           |Staten Island|Arden Heights          |Boro Zone   |
|6           |Staten Island|Arrochar/Fort Wadsworth|Boro Zone   |
|7           |Queens       |Astoria                |Boro Zone   |
|8           |Queens       |Astoria Park           |Boro Zone   |
|9           |Queens       |Auburndale             |Boro Zone   |
|10          |Queens       |Baisley Park           |Boro Zone   |
+------------+-------------+-----------------------+------------+
only showing top 10 rows

In [13]:
sample_with_zones.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+
|          79|          68|    

In [14]:
from pyspark.sql.functions import when, col

sample_with_zones = sample_with_zones.withColumn(
    "tip_percentage",
    when(col("fare_amount") > 0, (col("tip_amount") / col("fare_amount")) * 100).otherwise(0)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
sample_with_zones.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+---------------+------------+---------+--------------------+------------+------------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough|    pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|    tip_percentage|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+---------------+------------+---------+-------------------

In [16]:
cleaned_df = sample_with_zones.filter(
    (col("trip_distance") <= 100) & (col("fare_amount") <= 500)
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
cleaned_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+---------------+------------+---------+--------------------+------------+------------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough|    pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|    tip_percentage|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+---------------+------------+---------+-------------------

In [18]:
from pyspark.sql.functions import when

bucketed_df = cleaned_df.withColumn(
    "distance_bucket",
    when(col("trip_distance") < 1, "0-1 miles")
    .when((col("trip_distance") >= 1) & (col("trip_distance") < 5), "1-5 miles")
    .when((col("trip_distance") >= 5) & (col("trip_distance") < 10), "5-10 miles")
    .otherwise("10+ miles")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
bucketed_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+-----------------+---------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|   tip_percentage|distance_bucket|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+--------

In [20]:
from pyspark.sql.functions import expr

mapped_df = bucketed_df.withColumn(
    "payment_type_desc",
    expr("""
        CASE payment_type
            WHEN 1 THEN 'Credit Card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No Charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            WHEN 6 THEN 'Voided'
            ELSE 'Other'
        END
    """)
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
mapped_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+-----------------+---------------+-----------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|   tip_percentage|distance_bucket|payment_type_desc|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+--------

In [22]:
mapped_df.select("VendorID").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|VendorID|
+--------+
|       1|
|       3|
|       2|
|       4|
+--------+

In [23]:
mapped_df.select("VendorID").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|VendorID|
+--------+
|       1|
|       3|
|       2|
|       4|
+--------+

In [24]:
from pyspark.sql.functions import col

clean_sample_df = mapped_df.withColumn("RatecodeID", col("RatecodeID").cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
clean_sample_df.select("RatecodeID").distinct().orderBy("RatecodeID").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|RatecodeID|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|        99|
+----------+

In [26]:
from pyspark.sql.functions import when

clean_sample_df = clean_sample_df.withColumn(
    "ratecode_desc",
    when(col("RatecodeID") == 1, "Standard rate")
    .when(col("RatecodeID") == 2, "JFK")
    .when(col("RatecodeID") == 3, "Newark")
    .when(col("RatecodeID") == 4, "Nassau or Westchester")
    .when(col("RatecodeID") == 5, "Negotiated fare")
    .when(col("RatecodeID") == 6, "Group ride")
    .when(col("RatecodeID") == 99, "Unknown")
    .otherwise("Other")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+-----------------+---------------+-----------------+-------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|   tip_percentage|distance_bucket|payment_type_desc|ratecode_desc|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+---------

In [28]:
clean_sample_df.groupBy("VendorID").count().orderBy("VendorID").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+
|VendorID| count|
+--------+------+
|       1|405444|
|       2|586086|
|       3|    12|
|       4|  1373|
+--------+------+

In [29]:
from pyspark.sql.functions import col

# Keep only VendorID 1 and 2
clean_sample_df = clean_sample_df.filter((col("VendorID") == 1) | (col("VendorID") == 2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
clean_sample_df.select("VendorID").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|VendorID|
+--------+
|       1|
|       2|
+--------+

In [31]:
from pyspark.sql.functions import when, col

clean_sample_df = clean_sample_df.withColumn(
    "vendor_desc",
    when(col("VendorID") == 1, "Creative Mobile Technologies, LLC")
    .when(col("VendorID") == 2, "Curb Mobility, LLC")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+-----------------+---------------+-----------------+-------------+--------------------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|   tip_percentage|distance_bucket|payment_type_desc|ratecode_desc|         vendor_desc|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+--

In [33]:
from pyspark.sql.functions import to_date, date_format

# Pickup Date and Time
clean_sample_df = clean_sample_df.withColumn("pickup_date", to_date("tpep_pickup_datetime")) \
                                 .withColumn("pickup_time", date_format("tpep_pickup_datetime", "HH:mm:ss"))

# Dropoff Date and Time
clean_sample_df = clean_sample_df.withColumn("drop_date", to_date("tpep_dropoff_datetime")) \
                                 .withColumn("drop_time", date_format("tpep_dropoff_datetime", "HH:mm:ss"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+---------+------------+------------+---------+--------------------+------------+-----------------+---------------+-----------------+-------------+--------------------+-----------+-----------+----------+---------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year|  Borough| pickup_zone|service_zone|  Borough|        dropoff_zone|service_zone|   tip_percentage|distance_bucket|payment_type_desc|ratecode_desc|         vendor_desc|pickup_date|pickup_time| drop_date|drop_time|
+------------+------------+------------------+---------------+-----+----------+-----------

In [35]:
clean_sample_df.select("tpep_pickup_datetime", "pickup_date", "pickup_time",
                       "tpep_dropoff_datetime", "drop_date", "drop_time").show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-----------+---------------------+----------+---------+
|tpep_pickup_datetime|pickup_date|pickup_time|tpep_dropoff_datetime|drop_date |drop_time|
+--------------------+-----------+-----------+---------------------+----------+---------+
|2016-01-01 00:38:14 |2016-01-01 |00:38:14   |2016-01-01 00:52:44  |2016-01-01|00:52:44 |
|2016-01-01 00:16:05 |2016-01-01 |00:16:05   |2016-01-01 00:25:36  |2016-01-01|00:25:36 |
|2016-01-01 00:19:00 |2016-01-01 |00:19:00   |2016-01-01 00:22:41  |2016-01-01|00:22:41 |
|2016-01-01 00:42:25 |2016-01-01 |00:42:25   |2016-01-01 00:42:47  |2016-01-01|00:42:47 |
|2016-01-01 00:51:52 |2016-01-01 |00:51:52   |2016-01-01 01:02:37  |2016-01-01|01:02:37 |
+--------------------+-----------+-----------+---------------------+----------+---------+
only showing top 5 rows

In [36]:
clean_sample_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- service_zone: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- dropoff_zone: string (null

In [37]:
# Row count
row_count = clean_sample_df.count()

# Column count
column_count = len(clean_sample_df.columns)

print(f"Row count: {row_count}")
print(f"Column count: {column_count}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row count: 991530
Column count: 33

In [38]:
clean_sample_df.select("year").distinct().orderBy("year").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|year|
+----+
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
+----+

In [40]:
clean_sample_df.coalesce(1).write.mode("overwrite").parquet("s3://nycfinalp/sample-data-with-all-columns/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Found duplicate column(s) when inserting into s3://nycfinalp/sample-data-with-all-columns: `service_zone`, `borough`;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 844, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Found duplicate column(s) when inserting into s3://nycfinalp/sample-data-with-all-columns: `service_zone`, `borough`;'



In [41]:
# If two columns with same name exist, Spark won’t allow writing.
# Drop one of each duplicate manually, based on what you want to keep.

clean_sample_df = clean_sample_df.drop("service_zone").drop("borough")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+---------------------+------------+----------+------------+----+------------+--------------------+-----------------+---------------+-----------------+-------------+--------------------+-----------+-----------+----------+---------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year| pickup_zone|        dropoff_zone|   tip_percentage|distance_bucket|payment_type_desc|ratecode_desc|         vendor_desc|pickup_date|pickup_time| drop_date|drop_time|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+--------------------+---------------------+----

In [43]:
clean_sample_df.coalesce(1).write.mode("overwrite").parquet("s3://nycfinalp/sample-data-with-all-columns/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
clean_sample_df = clean_sample_df.drop("tpep_pickup_datetime", "tpep_dropoff_datetime")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
clean_sample_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+---------------------+------------+----------+------------+----+------------+--------------------+-----------------+---------------+-----------------+-------------+--------------------+-----------+-----------+----------+---------+
|DOLocationID|PULocationID|store_and_fwd_flag|passenger_count|extra|tip_amount|fare_amount|total_amount|mta_tax|trip_distance|VendorID|improvement_surcharge|payment_type|RatecodeID|tolls_amount|year| pickup_zone|        dropoff_zone|   tip_percentage|distance_bucket|payment_type_desc|ratecode_desc|         vendor_desc|pickup_date|pickup_time| drop_date|drop_time|
+------------+------------+------------------+---------------+-----+----------+-----------+------------+-------+-------------+--------+---------------------+------------+----------+------------+----+------------+--------------------+-----------------+---------------+-

In [46]:
clean_sample_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)
 |-- tip_percentage: double (nullable = true)
 |-- distance_bucket: string (nullable = false)
 |-- payment_type_desc: string (nullable = false)
 |-- ratecode_desc: string (nullable = false)
 |-- vendor_desc: string (null

In [47]:
# Row count
row_count = clean_sample_df.count()

# Column count
column_count = len(clean_sample_df.columns)

print(f"Row count: {row_count}")
print(f"Column count: {column_count}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row count: 991530
Column count: 27

In [48]:
clean_sample_df.coalesce(1).write.mode("overwrite").parquet("s3://nycfinalp/sample-data-final/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…